In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from collections import Counter

In [2]:
iris = datasets.load_iris()

In [3]:
# iris.data
# iris.target
# iris.feature_names

In [4]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['type'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


Сравним количество одинаковых значений для разных классов у признаков **sepal length (cm)** и **petal width (cm)**.

In [5]:
len(set(iris_df.loc[iris_df.type == 0, 'sepal length (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 1, 'sepal length (cm)'].values)) + \
len(set(iris_df.loc[iris_df.type == 1, 'sepal length (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 2, 'sepal length (cm)'].values)) + \
len(set(iris_df.loc[iris_df.type == 0, 'sepal length (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 2, 'sepal length (cm)'].values))

25

In [6]:
len(set(iris_df.loc[iris_df.type == 0, 'petal width (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 1, 'petal width (cm)'].values)) + \
len(set(iris_df.loc[iris_df.type == 1, 'petal width (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 2, 'petal width (cm)'].values)) + \
len(set(iris_df.loc[iris_df.type == 0, 'petal width (cm)'].values) & 
    set(iris_df.loc[iris_df.type == 2, 'petal width (cm)'].values))

5

Количество одинаковых значений у признака **sepal length (cm)** больше, чем у признака **petal width (cm)**, поэтому прогноз дерева решений по признаку **sepal length (cm)** будет хуже.

In [7]:
def h_ginni(l):
    return sum([(x / len(l))* (1 - x / len(l)) for x in Counter(l).values()])

In [8]:
def h_entropy(l):
    return -sum([(x / len(l)) * np.log2(x / len(l)) for x in Counter(l).values()])

In [9]:
class decision_tree_classification:
    """
    Реализует процесс построения дерева решений с помощью одномерного разбиения 
    Принимает в качестве параметров:
    1) значения одного признака
    2) значения классов
    3) функцию для вычисления критерия информативности
    4) глубину дерева решений
    """
    def __init__(self, x, y, f, n): 
        df = pd.DataFrame({'x': x, 'y': y}).sort_values(by='x')
        self.x = df.x.values
        self.y = df.y.values
        self.f = f
        self.n = n
    
    def node_select(self, y, f):
        """
        Разбивает массив y на подмножества по критерию информативности,
        вычисленному с помощью функции f
        """
        treshold1 = 0
        treshold2 = 0
        igr0 = 0
        hr = f(y)
        for i in range(len(y) - 1):
            for j in range(i + 1, len(y)):
                hr0 = f(y[:i])
                hr1 = f(y[i:j])
                hr2 = f(y[j:])
                igr = hr - (len(y[:i]) / len(y)) * hr0 - (len(y[i:j]) / len(y)) * hr1 - (len(y[j:]) / len(y)) * hr2
                if np.round(igr0, 4) <= np.round(igr, 4):
                    igr0 = igr
                    treshold1 = i
                    treshold2 = j
            if len(np.unique(y)) == 2 and i == 0:
                break
        return igr0, treshold1, treshold2
    
    def classifier(self):
        """
        Возвращает индексы узлов в отсортированном массиве значений признака, по которому
        строится дерево решений, полученных при заданной глубине дерева, либо когда функционал
        качества будет равен 0 для всех узлов
        """
        nodes = [self.y]
        nodes_temp = []
        end_nodes = []
        threshold_prev = [0]
        threshold = []
        stop_criteria = 0

        for i in range(self.n):
            for j, nodes[j] in zip(range(len(nodes)), nodes):
                igr, t1, t2 = self.node_select(nodes[j], self.f)
                if igr != 0:
                    threshold.append(threshold_prev[j])
                    if t1 != 0:
                        threshold.append(threshold_prev[j] + t1)
                    threshold.append(threshold_prev[j] + t2)
                    if t1 != 0:
                        nodes_temp.append(nodes[j][:t1])
                    nodes_temp.append(nodes[j][t1:t2])
                    nodes_temp.append(nodes[j][t2:])
                    if i == self.n - 1 and len(nodes[j]) > 1:
                        end_nodes.append(threshold_prev[j])
                else:
                    end_nodes.append(threshold_prev[j])

                stop_criteria += igr

            if stop_criteria == 0:
                break

            stop_criteria = 0
            threshold_prev = threshold
            threshold = []
            nodes = nodes_temp
            nodes_temp = []

        return sorted(list(set(end_nodes)))
    
    def prediction(self, x_test):
        """
        Возвращает массив с предсказанными метками класса
        """
        predicted = []
        nodes = self.classifier()
        for el in x_test:
            for i in range(len(nodes)):
                if i != 0:
                    if i != len(nodes) - 1:
                        if self.x[nodes[i]] <= el < self.x[nodes[i + 1]]:
                            predicted.append(self.y[nodes[i + 1] - 1])
                    else:
                        if el >= self.x[nodes[i]]:
                            predicted.append(self.y[nodes[i]])
                else:
                    if el < self.x[nodes[i + 1]]:
                        predicted.append(self.y[nodes[i + 1] - 1])
        return predicted

Сравним точность модели для разных признаков.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=13)

In [11]:
dt_class = decision_tree_classification(X_train[:, 3], y_train, h_ginni, 5)

In [12]:
predicted1 = dt_class.prediction(X_test[:, 3])

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Decision tree metrics for 'petal width' attribute")
print('accuracy: %.3f' % accuracy_score(y_test, predicted1))
print('precision: %.3f' % precision_score(y_test, predicted1, average='weighted'))
print('recall: %.3f' % recall_score(y_test, predicted1, average='weighted'))

Decision tree metrics for 'petal width' attribute
accuracy: 0.967
precision: 0.970
recall: 0.967


In [14]:
dt_class = decision_tree_classification(X_train[:, 0], y_train, h_ginni, 5)

In [15]:
predicted2 = dt_class.prediction(X_test[:, 0])

In [16]:
print("Decision tree metrics for 'sepal length' attribute")
print('accuracy: %.3f' % accuracy_score(y_test, predicted2))
print('precision: %.3f' % precision_score(y_test, predicted2, average='weighted'))
print('recall: %.3f' % recall_score(y_test, predicted2, average='weighted'))

Decision tree metrics for 'sepal length' attribute
accuracy: 0.500
precision: 0.576
recall: 0.500
